In [1]:
## Seismic Analysis Impact on PCI - Control Population
## By: Fervis Lauan
## Date: 09-29-17
##----------------------------------------------------

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import requests
from census import Census
from us import states


In [2]:
##################################
# Get Control Population
##################################


yr=2012
ctr=1
while yr<=2013:
    try:
        c = Census("85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98", year=yr)
        census_data = c.acs5.get(("NAME",
                                          "B01003_001E", 
                                          "B01002_001E",
                                          "B19013_001E",
                                          "B19301_001E",
                                          "B17001_002E",
                                          "B23025_002E",
                                          "B23025_005E",
                                          "B25077_001E",
                                          "B25064_001E",
                                          "C24010_009E",
                                          "C24010_016E",
                                          "C24010_019E",
                                          "C24010_022E",
                                          "C24010_023E",
                                          "C24010_025E",
                                          "C24010_030E",
                                          "C24010_032E",
                                          "C24010_045E",
                                          "C24010_052E",
                                          "C24010_055E",
                                          "C24010_058E",
                                          "C24010_059E",
                                          "C24010_061E",
                                          "C24010_066E",
                                          "C24010_068E",                         
                                 ), {'for': 'zip code tabulation area:*'})
        # Convert to DataFrame
        census_pd = pd.DataFrame(census_data)
        census_pd["Year"]=yr
        # Column Reordering
        census_pd = census_pd.rename(columns={"B01003_001E": "Population", 
                                              "B01002_001E": "Median Age",
                                              "B19013_001E": "Household Income",
                                              "B19301_001E": "Per Capita Income",
                                              "B17001_002E": "Poverty Count",
                                              "B23025_002E": "Employment Count",
                                              "B23025_005E": "Unemployment Count",
                                              "B25077_001E": "Median Home Value",
                                              "B25064_001E": "Median Gross Rent",                                      
                                              "C24010_009E": "Emp Male Arch Engnr",
                                              "C24010_016E": "Emp Male Health Prac",
                                              "C24010_019E": "Emp Male Service Occ",
                                              "C24010_022E": "Emp Male Fire Prev",
                                              "C24010_023E": "Emp Male Law Enfrcmnt",
                                              "C24010_025E": "Emp Male BldgGrnd Cleaning",
                                              "C24010_030E": "Emp Male Ntrl Rsrces Const",
                                              "C24010_032E": "Emp Male Const Extrctn",                                      
                                              "C24010_045E": "Emp FMale Arch Engnr",
                                              "C24010_052E": "Emp FMale Health Prac",
                                              "C24010_055E": "Emp FMale Service Occ",
                                              "C24010_058E": "Emp FMale Fire Prev",
                                              "C24010_059E": "Emp FMale Law Enfrcmnt",
                                              "C24010_061E": "Emp FMale BldgGrnd Cleaning",
                                              "C24010_066E": "Emp FMale Ntrl Rsrces Const",
                                              "C24010_068E": "Emp FMale Const Extrctn",                                                                          
                                              "NAME": "Name",                                       
                                              "zip code tabulation area": "Zip",
                                              "Year":"Year"})

        census_pd["Emp Arch Engnr"]=census_pd["Emp Male Arch Engnr"]+census_pd["Emp FMale Arch Engnr"]
        census_pd["Emp Health Prac"]=census_pd["Emp Male Health Prac"]+census_pd["Emp FMale Health Prac"]
        census_pd["Emp Service Occ"]=census_pd["Emp Male Service Occ"]+census_pd["Emp FMale Service Occ"]
        census_pd["Emp Fire Prev"]=census_pd["Emp Male Fire Prev"]+census_pd["Emp FMale Fire Prev"]
        census_pd["Emp Law Enfrcmnt"]=census_pd["Emp Male Law Enfrcmnt"]+census_pd["Emp FMale Law Enfrcmnt"]
        census_pd["Emp BldgGrnd Cleaning"]=census_pd["Emp Male BldgGrnd Cleaning"]+census_pd["Emp FMale BldgGrnd Cleaning"]
        census_pd["Emp Ntrl Rsrces Const"]=census_pd["Emp Male Ntrl Rsrces Const"]+census_pd["Emp FMale Ntrl Rsrces Const"]
        census_pd["Emp Const Extrctn"]=census_pd["Emp Male Const Extrctn"]+census_pd["Emp FMale Const Extrctn"]

        # Add in Poverty Rate (Poverty Count / Population)
        census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

        # Add in Employment Rate (Employment Count / Population)
        census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"].astype(int) / census_pd["Population"].astype(int)

        # Final DataFrame
        census_pd = census_pd[["Year",                       
                               "Name", 
                               "Zip",
                               "Population", 
                               "Median Age", 
                               "Household Income",
                               "Per Capita Income", 
                               "Poverty Count", 
                               "Poverty Rate",
                               "Employment Count", 
                               "Unemployment Rate",
                               "Median Home Value",
                               "Median Gross Rent",
                               "Emp Arch Engnr",
                               "Emp Health Prac",
                               "Emp Service Occ",
                               "Emp Fire Prev",
                               "Emp Law Enfrcmnt",
                               "Emp BldgGrnd Cleaning",
                               "Emp Ntrl Rsrces Const",
                               "Emp Const Extrctn"
                               ]]                 

        if ctr==1:
            print("Creating from year "+str(yr)+"...")
            census_pd_ctrl=census_pd
        else:
            print("appending year "+str(yr)+"...")
            census_pd_ctrl=census_pd_ctrl.append(census_pd)
        ctr+=1
    except:
        print("No Data")
    yr+=1
    

Creating from year 2012...
appending year 2013...


In [3]:
census_pd_ctrl2012=census_pd_ctrl.loc[census_pd_ctrl["Year"]==2012]
census_pd_ctrl2013=census_pd_ctrl.loc[census_pd_ctrl["Year"]==2013]

In [4]:
census_pd_ctrlmrg=pd.merge(census_pd_ctrl2013, census_pd_ctrl2012, on="Zip")
census_pd_ctrlmrg

Year_x       Name_x    Zip  Population_x  Median Age_x  \
0        2013  ZCTA5 00601  00601       18450.0          36.6   
1        2013  ZCTA5 00602  00602       41302.0          38.6   
2        2013  ZCTA5 00603  00603       53683.0          38.9   
3        2013  ZCTA5 00606  00606        6591.0          37.3   
4        2013  ZCTA5 00610  00610       28963.0          39.2   
5        2013  ZCTA5 00612  00612       68055.0          38.5   
6        2013  ZCTA5 00616  00616       10183.0          40.9   
7        2013  ZCTA5 00617  00617       24879.0          36.2   
8        2013  ZCTA5 00622  00622        6165.0          42.0   
9        2013  ZCTA5 00623  00623       44704.0          39.7   
10       2013  ZCTA5 00624  00624       25340.0          34.2   
11       2013  ZCTA5 00627  00627       34806.0          37.9   
12       2013  ZCTA5 00631  00631        1747.0          43.9   
13       2013  ZCTA5 00637  00637       24996.0          38.8   
14       2013  ZCTA5 00638  00638       18742.0          37.2   
15       2013  ZCTA5 00641  00641       30401.0          38.7   
16       2013  ZCTA5 00646  00646       38303.0          37.5   
17       2013  ZCTA5 00647  00647        5640.0          41.3   
18       2013  ZCTA5 00650  00650       14969.0          36.2   
19       2013  ZCTA5 00652  00652        3461.0          40.1   
20       2013  ZCTA5 00653  00653       13461.0          36.6   
21       2013  ZCTA5 00656  00656       21172.0          36.5   
22       2013  ZCTA5 00659  00659       41932.0          37.9   
23       2013  ZCTA5 00660  00660       17151.0          42.4   
24       2013  ZCTA5 00662  00662       43535.0          38.4   
25       2013  ZCTA5 00664  00664       16508.0          33.9   
26       2013  ZCTA5 00667  00667       25797.0          39.6   
27       2013  ZCTA5 00669  00669       28984.0          37.7   
28       2013  ZCTA5 00670  00670        9689.0          37.7   
29       2013  ZCTA5 00674  00674       43848.0          37.9   
...       ...          ...    ...           ...           ...   
33090    2013  ZCTA5 99786  99786         290.0          25.0   
33091    2013  ZCTA5 99788  99788          65.0          21.5   
33092    2013  ZCTA5 99789  99789         405.0          24.4   
33093    2013  ZCTA5 99790  99790          43.0          50.6   
33094    2013  ZCTA5 99791  99791         276.0          23.0   
33095    2013  ZCTA5 99801  99801       29639.0          37.4   
33096    2013  ZCTA5 99820  99820         457.0          45.2   
33097    2013  ZCTA5 99824  99824        2242.0          36.0   
33098    2013  ZCTA5 99825  99825          91.0          54.6   
33099    2013  ZCTA5 99826  99826         538.0          47.8   
33100    2013  ZCTA5 99827  99827        2656.0          44.0   
33101    2013  ZCTA5 99829  99829         775.0          42.7   
33102    2013  ZCTA5 99830  99830         487.0          42.5   
33103    2013  ZCTA5 99832  99832          64.0          50.7   
33104    2013  ZCTA5 99833  99833        3277.0          42.4   
33105    2013  ZCTA5 99835  99835        8945.0          38.5   
33106    2013  ZCTA5 99836  99836          42.0          27.0   
33107    2013  ZCTA5 99840  99840        1080.0          44.3   
33108    2013  ZCTA5 99841  99841         106.0          51.7   
33109    2013  ZCTA5 99901  99901       13651.0          38.7   
33110    2013  ZCTA5 99903  99903          30.0          63.7   
33111    2013  ZCTA5 99918  99918         130.0          52.8   
33112    2013  ZCTA5 99919  99919         528.0          43.1   
33113    2013  ZCTA5 99921  99921        2104.0          38.0   
33114    2013  ZCTA5 99922  99922         364.0          40.0   
33115    2013  ZCTA5 99923  99923          81.0          64.3   
33116    2013  ZCTA5 99925  99925         668.0          46.4   
33117    2013  ZCTA5 99926  99926        1630.0          31.4   
33118    2013  ZCTA5 99927  99927         111.0          59.4   
33119    2013  ZCTA5 99929  99929        2

In [5]:
census_pd_ctrlmrg=census_pd_ctrlmrg.loc[(census_pd_ctrlmrg["Population_x"]>=11000) & (census_pd_ctrlmrg["Population_x"]<=50000)]
census_pd_ctrlmrg

Year_x       Name_x    Zip  Population_x  Median Age_x  \
0        2013  ZCTA5 00601  00601       18450.0          36.6   
1        2013  ZCTA5 00602  00602       41302.0          38.6   
4        2013  ZCTA5 00610  00610       28963.0          39.2   
7        2013  ZCTA5 00617  00617       24879.0          36.2   
9        2013  ZCTA5 00623  00623       44704.0          39.7   
10       2013  ZCTA5 00624  00624       25340.0          34.2   
11       2013  ZCTA5 00627  00627       34806.0          37.9   
13       2013  ZCTA5 00637  00637       24996.0          38.8   
14       2013  ZCTA5 00638  00638       18742.0          37.2   
15       2013  ZCTA5 00641  00641       30401.0          38.7   
16       2013  ZCTA5 00646  00646       38303.0          37.5   
18       2013  ZCTA5 00650  00650       14969.0          36.2   
20       2013  ZCTA5 00653  00653       13461.0          36.6   
21       2013  ZCTA5 00656  00656       21172.0          36.5   
22       2013  ZCTA5 00659  00659       41932.0          37.9   
23       2013  ZCTA5 00660  00660       17151.0          42.4   
24       2013  ZCTA5 00662  00662       43535.0          38.4   
25       2013  ZCTA5 00664  00664       16508.0          33.9   
26       2013  ZCTA5 00667  00667       25797.0          39.6   
27       2013  ZCTA5 00669  00669       28984.0          37.7   
29       2013  ZCTA5 00674  00674       43848.0          37.9   
30       2013  ZCTA5 00676  00676       39916.0          35.8   
31       2013  ZCTA5 00677  00677       15108.0          41.3   
32       2013  ZCTA5 00678  00678       27229.0          37.5   
34       2013  ZCTA5 00682  00682       32509.0          35.0   
35       2013  ZCTA5 00683  00683       33442.0          40.6   
36       2013  ZCTA5 00685  00685       41947.0          39.7   
37       2013  ZCTA5 00687  00687       32609.0          33.8   
38       2013  ZCTA5 00688  00688       13340.0          40.1   
40       2013  ZCTA5 00692  00692       37618.0          35.0   
...       ...          ...    ...           ...           ...   
32856    2013  ZCTA5 99336  99336       48749.0          30.7   
32857    2013  ZCTA5 99337  99337       30233.0          39.1   
32858    2013  ZCTA5 99338  99338       12244.0          38.5   
32861    2013  ZCTA5 99344  99344       16898.0          26.3   
32867    2013  ZCTA5 99350  99350       13636.0          33.7   
32868    2013  ZCTA5 99352  99352       27833.0          39.6   
32869    2013  ZCTA5 99353  99353       13652.0          40.1   
32870    2013  ZCTA5 99354  99354       22294.0          36.4   
32876    2013  ZCTA5 99362  99362       41513.0          38.0   
32881    2013  ZCTA5 99403  99403       19917.0          43.7   
32882    2013  ZCTA5 99501  99501       16521.0          38.6   
32883    2013  ZCTA5 99502  99502       24059.0          33.7   
32884    2013  ZCTA5 99503  99503       14042.0          34.9   
32885    2013  ZCTA5 99504  99504       40824.0          32.4   
32888    2013  ZCTA5 99507  99507       39015.0          32.7   
32889    2013  ZCTA5 99508  99508       36534.0          29.7   
32892    2013  ZCTA5 99515  99515       22821.0          34.8   
32893    2013  ZCTA5 99516  99516       20781.0          42.0   
32894    2013  ZCTA5 99517  99517       16442.0          34.1   
32926    2013  ZCTA5 99577  99577       27158.0          33.2   
32947    2013  ZCTA5 99611  99611       15779.0          39.0   
32951    2013  ZCTA5 99615  99615       13009.0          31.5   
32973    2013  ZCTA5 99645  99645       26202.0          35.1   
32996    2013  ZCTA5 99669  99669       14572.0          35.6   
33020    2013  ZCTA5 99701  99701       19067.0          31.7   
33024    2013  ZCTA5 99705  99705       24098.0          29.8   
33026    2013  ZCTA5 99709  99709       31376.0          35.1   
33027    2013  ZCTA5 99712  99712       11945.0          38.7   
33095    2013  ZCTA5 99801  99801       29639.0          37.4   
33109    2013  ZCTA5 99901  99901       13

In [6]:
census_pd_ctrlsample=census_pd_ctrlmrg.sample(1000)


In [7]:
census_pd_ctrlsample

Year_x       Name_x    Zip  Population_x  Median Age_x  \
9406     2013  ZCTA5 29585  29585       13356.0          55.4   
27173    2013  ZCTA5 78248  78248       14405.0          42.2   
6844     2013  ZCTA5 22315  22315       27231.0          36.2   
16244    2013  ZCTA5 49048  49048       25434.0          34.9   
1048     2013  ZCTA5 04072  04072       18647.0          42.3   
6580     2013  ZCTA5 21236  21236       38166.0          37.2   
30056    2013  ZCTA5 90049  90049       35310.0          41.4   
1069     2013  ZCTA5 04105  04105       11260.0          47.3   
27253    2013  ZCTA5 78504  78504       49936.0          31.4   
6544     2013  ZCTA5 21158  21158       19873.0          41.2   
18152    2013  ZCTA5 54501  54501       19965.0          46.6   
27976    2013  ZCTA5 80525  80525       48207.0          35.7   
6678     2013  ZCTA5 21704  21704       14222.0          36.1   
29172    2013  ZCTA5 85298  85298       22106.0          36.5   
26958    2013  ZCTA5 77662  77662       26765.0          38.8   
1838     2013  ZCTA5 06437  06437       22377.0          47.6   
7228     2013  ZCTA5 23601  23601       24907.0          36.4   
10164    2013  ZCTA5 31605  31605       21588.0          28.2   
510      2013  ZCTA5 02189  02189       14609.0          42.5   
1720     2013  ZCTA5 06071  06071       11599.0          42.3   
16015    2013  ZCTA5 48506  48506       29071.0          37.7   
8407     2013  ZCTA5 27263  27263       22058.0          38.8   
13161    2013  ZCTA5 40505  40505       26212.0          38.6   
30586    2013  ZCTA5 92618  92618       18617.0          34.9   
29333    2013  ZCTA5 85748  85748       18275.0          45.4   
14167    2013  ZCTA5 44026  44026       11218.0          46.9   
15260    2013  ZCTA5 46806  46806       24414.0          28.7   
29701    2013  ZCTA5 88007  88007       26957.0          38.1   
24714    2013  ZCTA5 71909  71909       15043.0          66.9   
8954     2013  ZCTA5 28539  28539       15202.0          27.7   
...       ...          ...    ...           ...           ...   
2013     2013  ZCTA5 07063  07063       14552.0          33.4   
16839    2013  ZCTA5 50208  50208       20675.0          42.6   
12338    2013  ZCTA5 38108  38108       18788.0          34.1   
29174    2013  ZCTA5 85302  85302       37149.0          35.0   
4484     2013  ZCTA5 15238  15238       12654.0          50.1   
7286     2013  ZCTA5 23860  23860       30917.0          36.8   
6246     2013  ZCTA5 20110  20110       45480.0          33.1   
28806    2013  ZCTA5 84040  84040       23524.0          32.6   
8448     2013  ZCTA5 27370  27370       14002.0          45.0   
12707    2013  ZCTA5 39042  39042       33776.0          39.1   
4431     2013  ZCTA5 15122  15122       20123.0          44.7   
27428    2013  ZCTA5 78840  78840       48325.0          32.1   
20551    2013  ZCTA5 60194  60194       20322.0          37.5   
9575     2013  ZCTA5 30009  30009       14584.0          38.3   
31580    2013  ZCTA5 95833  95833       38264.0          31.1   
15223    2013  ZCTA5 46750  46750       27432.0          39.7   
9599     2013  ZCTA5 30045  30045       33063.0          35.8   
11511    2013  ZCTA5 35757  35757       13997.0          38.1   
30500    2013  ZCTA5 92344  92344       20116.0          31.6   
17748    2013  ZCTA5 53154  53154       34580.0          37.7   
26760    2013  ZCTA5 77327  77327       20829.0          35.2   
30969    2013  ZCTA5 94070  94070       29531.0          42.0   
6229     2013  ZCTA5 20016  20016       34144.0          37.8   
16885    2013  ZCTA5 50263  50263       17390.0          32.1   
24162    2013  ZCTA5 70126  70126       23327.0          34.9   
25983    2013  ZCTA5 75216  75216       49342.0          34.4   
1865     2013  ZCTA5 06482  06482       11218.0          42.2   
29975    2013  ZCTA5 89460  89460       13338.0          44.5   
27963    2013  ZCTA5 80503  80503       31302.0          42.8   
30859    2013  ZCTA5 93619  93619       29

In [8]:
for index,row in census_pd_ctrlsample.iterrows():
    pcchg=(row["Per Capita Income_x"]-row["Per Capita Income_y"])/row["Per Capita Income_y"]
    census_pd_ctrlsample.loc[index,"PCI Change"]=pcchg
    


In [9]:
census_pd_ctrlsample.to_csv("./Resources/seismic_ctrl_pop.csv")

    
    

In [10]:
census_pd_ctrl_ttest=census_pd_ctrlsample[["Zip","PCI Change"]]
census_pd_ctrl_ttest

Zip  PCI Change
9406   29585   -0.062074
27173  78248    0.007755
6844   22315    0.012051
16244  49048   -0.028741
1048   04072    0.038603
6580   21236    0.001031
30056  90049   -0.045848
1069   04105    0.004361
27253  78504   -0.052511
6544   21158    0.031433
18152  54501    0.058915
27976  80525    0.010491
6678   21704    0.039551
29172  85298    0.006284
26958  77662    0.029920
1838   06437    0.033070
7228   23601   -0.031314
10164  31605   -0.024800
510    02189   -0.042112
1720   06071   -0.027464
16015  48506    0.035736
8407   27263   -0.029227
13161  40505    0.005054
30586  92618    0.005025
29333  85748    0.018742
14167  44026    0.016941
15260  46806   -0.039769
29701  88007    0.050732
24714  71909    0.035162
8954   28539    0.030660
...      ...         ...
2013   07063   -0.056273
16839  50208    0.042233
12338  38108    0.071060
29174  85302   -0.076913
4484   15238    0.037528
7286   23860    0.024758
6246   20110   -0.000203
28806  84040    0.031373
8448   27370   -0.013803
12707  39042    0.016468
4431   15122    0.040496
27428  78840    0.018715
20551  60194    0.038446
9575   30009   -0.050570
31580  95833    0.017752
15223  46750    0.027461
9599   30045   -0.026055
11511  35757   -0.052732
30500  92344   -0.024223
17748  53154    0.013445
26760  77327    0.006491
30969  94070   -0.041032
6229   20016   -0.023795
16885  50263    0.014473
24162  70126   -0.036505
25983  75216   -0.063847
1865   06482   -0.016550
29975  89460   -0.044914
27963  80503   -0.000713
30859  93619    0.005996

[1000 rows x 2 columns]

In [11]:
census_pd_ctrl_ttest.to_csv("./Resources/seismic_ctrl_pop_ttest.csv")